In [1]:
# Proof-of-concept of our algorithm from "TSP Escapes the O(2^n n^2) Curse".
# We provide the standard DP as `tsp_dp` and ours as `tsp_min_plus`.
# (c) Mihail Stoian, May 5, 2024

import math, random, numpy as np

def generate_cost_matrix(n):
  c = np.full((n, n), math.inf) 
  for i in range(n):
    for j in range(n):
      if i != j:
        c[i][j] = random.randint(1, 100)
  return c

# The standard DP running in time O(2^n n^2).
def tsp_dp(c):
  n = c.shape[0]
  dp = np.full((2**n, n), math.inf)

  dp[1 << 0][0] = 0
  for mask in range(1, 2**n):
    for k in range(n):
      if (mask >> k) & 1:
        for j in range(n):
          if (mask >> j) & 1 and j != k:
            dp[mask][k] = min(dp[mask][k], dp[mask ^ (1 << k)][j] + c[j][k])
  ans = math.inf
  for k in range(n):
    ans = min(ans, dp[(1 << n) - 1][k] + c[k][0])
  return ans

# Naive min-plus product. For demonstration purposes: O(n^3). In theory: O(n^3 / 2^\Omega(\sqrt{\log n})).
def min_plus_prod(A, B):
  assert A.shape[1] == B.shape[0]
  ret = np.full((A.shape[0], B.shape[1]), math.inf)
  for i in range(A.shape[0]):
    for j in range(B.shape[1]):
      for k in range(A.shape[1]):
        ret[i][j] = min(ret[i][j], A[i][k] + B[k][j])
  return ret

# Our algorithm.
def tsp_min_plus(c):
  n = c.shape[0]
  dp = np.full((2**n, n), math.inf)

  # Preprocess the subsets of a given cardinality.
  partition = {}
  for i in range(1, n + 1):
    partition[i] = []
  for subset in range(1, 2**n):
    partition[bin(subset).count('1')].append(subset)

  # Return all batches of size `n`. It could be that the last batch is not complete.
  def batchify(size):
    index = 0
    while index < len(partition[size]):
      yield np.asarray(partition[size][index : index + n])
      index += n

  dp[1 << 0][0] = 0
  for l in range(2, n + 1):
    for batch in batchify(l - 1):
      # Takes O(n^3 / 2^\Omega(\sqrt{\log n})) using the fastest algorithm due to Williams.
      p = min_plus_prod(dp[batch], c)

      # Iterate all cells of the new matrix and fill the corresponding dp-entry. Takes O(n^2).
      for i, subset in enumerate(batch):
        for k in range(n):
          if ((subset >> k) & 1) == 0:
            dp[subset | (1 << k)][k] = min(dp[subset | (1 << k)][k], p[i][k])

  # Return solution.
  ans = math.inf
  for k in range(n):
    ans = min(ans, dp[(1 << n) - 1][k] + c[k][0])
  return ans

for n in range(5, 16):
  c = generate_cost_matrix(n)
  ans1 = tsp_dp(c)
  ans2 = tsp_min_plus(c)
  print(f'[check] n={n} | ans1={ans1} vs ans2={ans2}')
  assert ans1 == ans2

[check] n=5 | ans1=159.0 vs ans2=159.0
[check] n=6 | ans1=183.0 vs ans2=183.0
[check] n=7 | ans1=146.0 vs ans2=146.0
[check] n=8 | ans1=104.0 vs ans2=104.0
[check] n=9 | ans1=176.0 vs ans2=176.0
[check] n=10 | ans1=154.0 vs ans2=154.0
[check] n=11 | ans1=152.0 vs ans2=152.0
[check] n=12 | ans1=193.0 vs ans2=193.0
[check] n=13 | ans1=196.0 vs ans2=196.0
[check] n=14 | ans1=209.0 vs ans2=209.0
[check] n=15 | ans1=186.0 vs ans2=186.0
